# Exploring Spark with Pandas


Using pandas examples, convert the analysis to pyspark. This is useful if you discover your data grows too large for your tooling.

The purpose of this notebook is to familiarise yourself you the pyspark API. You are welcome to use the R version of this if you wish. As long as you are able to obtain the correct results. We will be using python in this notebook as it is quite widely used through data science and the community is very large.



#### Firstly, let's get our spark session

In [2]:
!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=1922fa2b35f884ed6c3e1c077d0e3f6209f6861c9ff90ed778af438691d4ec8b
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
import pandas as pd
spark = SparkSession.builder.appName('panda-and-spark').getOrCreate()

### Overview


* Joining two dataframes/data sets
* Simple aggregations
* Persisting

#### JOIN: Pandas

We won't use this more in this notebook, but observe how the joins work.

We what happens if you change from the default inner join to outer joins.

In [4]:
customer_raw = [(1, 'bob', 3462543658686),
           (2, 'rob', 9087567565439),
           (3, 'tim', 5436586999467),
           (4, 'tom', 8349756853250)]

customer_cols = ['id', 'name', 'credit_card_number']



orders_raw = [(1, 'ketchup', 'bob', 1.20),
           (2, 'rutabaga', 'bob', 3.35),
           (3, 'fake vegan meat', 'rob', 13.99),
           (4, 'cheesey poofs', 'tim', 3.99),
           (5, 'ice cream', 'tim', 4.95),
           (6, 'protein powder', 'tom', 49.95)]

orders_cols = ['id', 'product_name', 'customer', 'price']

In [5]:
customer_df = pd.DataFrame(customer_raw, columns=customer_cols)
orders_df = pd.DataFrame(orders_raw, columns=orders_cols)

customer_df

joined_df = pd.merge(customer_df, orders_df, how='inner', left_on='name', right_on='customer')
joined_df

## For self study. What happens if (4, 'tom', 8349756853250) in valuesA becomes (4, 'tod', 8349756853250)
## How do the results change?
## More sensibly; what if customers have not made any orders but we still require them in the result set?

,id_x,name,credit_card_number,id_y,product_name,customer,price
0,1,bob,3462543658686,1,ketchup,bob,1.20
1,1,bob,3462543658686,2,rutabaga,bob,3.35
2,2,rob,9087567565439,3,fake vegan meat,rob,13.99
3,3,tim,5436586999467,4,cheesey poofs,tim,3.99
4,3,tim,5436586999467,5,ice cream,tim,4.95
5,4,tom,8349756853250,6,protein powder,tom,49.95


#### JOIN: Spark

In [6]:
customersDF = spark.createDataFrame(customer_raw, customer_cols)

ordersDF = spark.createDataFrame(orders_raw, orders_cols)

# Show tables
customersDF.show()
ordersDF.show()

+---+----+------------------+
| id|name|credit_card_number|
+---+----+------------------+
|  1| bob|     3462543658686|
|  2| rob|     9087567565439|
|  3| tim|     5436586999467|
|  4| tom|     8349756853250|
+---+----+------------------+

+---+---------------+--------+-----+
| id|   product_name|customer|price|
+---+---------------+--------+-----+
|  1|        ketchup|     bob|  1.2|
|  2|       rutabaga|     bob| 3.35|
|  3|fake vegan meat|     rob|13.99|
|  4|  cheesey poofs|     tim| 3.99|
|  5|      ice cream|     tim| 4.95|
|  6| protein powder|     tom|49.95|
+---+---------------+--------+-----+



In [7]:
joinedDF = customersDF.join(ordersDF, customersDF.name == ordersDF.customer)
joinedDF.show()

+---+----+------------------+---+---------------+--------+-----+
| id|name|credit_card_number| id|   product_name|customer|price|
+---+----+------------------+---+---------------+--------+-----+
|  1| bob|     3462543658686|  1|        ketchup|     bob|  1.2|
|  1| bob|     3462543658686|  2|       rutabaga|     bob| 3.35|
|  2| rob|     9087567565439|  3|fake vegan meat|     rob|13.99|
|  3| tim|     5436586999467|  4|  cheesey poofs|     tim| 3.99|
|  3| tim|     5436586999467|  5|      ice cream|     tim| 4.95|
|  4| tom|     8349756853250|  6| protein powder|     tom|49.95|
+---+----+------------------+---+---------------+--------+-----+



## Simple Aggregations

Now let's explore simple aggregations. You will be using these often when doing exploratory work in big data. Remember, the intention here is that you grow familiar with the way the API works, and how to translate inquiries into that API.

> _How much did each person spend?_

In [8]:
joined_df.groupby('name').agg({"price": ["sum"]})

,price
,sum
name,
bob,4.55
rob,13.99
tim,8.94
tom,49.95


In [9]:
import pyspark.sql.functions as f

joinedDF.groupby('name').agg(f.sum('price').alias('total')).show()

+----+-----------------+
|name|            total|
+----+-----------------+
| bob|             4.55|
| rob|            13.99|
| tim|8.940000000000001|
| tom|            49.95|
+----+-----------------+



Let's use bigger data
  * NYC crash data

In [10]:
# save to the filesystem to prevent another load
! curl -o rows.csv https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  427M    0  427M    0     0  3770k      0 --:--:--  0:01:56 --:--:-- 2528k


In [11]:
import pandas as pd
nyc_df = pd.read_csv('rows.csv')

<ipython-input-11-a70f860f02a9>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  nyc_df = pd.read_csv('rows.csv')


In [15]:
# number or rows

print(len(nyc_df))

# this is quite large so we will work with a sample while we experiment in pandas as least.

2118226


We'll take a random sample at 20% of the original data

In [16]:
nyc_small = nyc_df.sample(frac=0.2, replace=False, random_state=1)

In [17]:
# we are also going to limit the columns to those we are going to work with

nyc_small = nyc_small[['CRASH DATE', 'CONTRIBUTING FACTOR VEHICLE 1',
                       'BOROUGH', 'VEHICLE TYPE CODE 1',
                       'NUMBER OF PERSONS INJURED']]

In [18]:
nyc_small.head(2)

,CRASH DATE,CONTRIBUTING FACTOR VEHICLE 1,BOROUGH,VEHICLE TYPE CODE 1,NUMBER OF PERSONS INJURED
430841,05/05/2019,Unsafe Speed,STATEN ISLAND,Station Wagon/Sport Utility Vehicle,0.0
1779911,04/15/2013,Other Electronic Device,QUEENS,PASSENGER VEHICLE,1.0


Now, let's create the pyspark dataframe. Now we two frames with the same content
  * nyc_small: pandas
  * sdf_small: pyspark

In [19]:
from pyspark.sql import SQLContext


# there are nan's in the frame with strings, and spark can't 'infer the schema', so we have to help it out
# by replacing them with empty strings and forcing the column to be a string

sdf_small = SQLContext(spark).createDataFrame(nyc_small.fillna('').astype('str'))


# Lets check the schema quickly

print(sdf_small.schema)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


StructType([StructField('CRASH DATE', StringType(), True), StructField('CONTRIBUTING FACTOR VEHICLE 1', StringType(), True), StructField('BOROUGH', StringType(), True), StructField('VEHICLE TYPE CODE 1', StringType(), True), StructField('NUMBER OF PERSONS INJURED', StringType(), True)])


# Questions

Answer the following questions by porting the pandas code to the Spark API



# Question 1


> On what day do most crashes occcur?

In [20]:
# Pandas
nyc_small.groupby('CRASH DATE')['CRASH DATE'].count().sort_values(ascending=False).head(5)

,CRASH DATE
CRASH DATE,
01/21/2014,246
02/03/2014,212
01/18/2015,209
05/19/2017,199
12/15/2017,198


In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, to_date, dayofweek


spark = SparkSession.builder.appName('CrashAnalysis').getOrCreate()

# Load the downloaded CSV file into a PySpark DataFrame
df = spark.read.option("header", "true").csv("rows.csv")


df = df.withColumn('CRASH DATE', to_date(df['CRASH DATE'], 'MM/dd/yyyy'))

df = df.withColumn('Day_of_Week', dayofweek('CRASH DATE'))

crashes_by_day = df.groupBy('Day_of_Week').agg(count('Day_of_Week').alias('count')).orderBy(col('count').desc())

# Display the result
crashes_by_day.show(5)


+-----------+------+
|Day_of_Week| count|
+-----------+------+
|          6|337359|
|          5|316355|
|          3|311635|
|          4|309487|
|          2|302556|
+-----------+------+
only showing top 5 rows



# Question 2

> _Where do most crashes occur?_

In [ ]:
nyc_small.groupby('BOROUGH')['BOROUGH'].count().sort_values(ascending=False).head(5)

In [22]:
## Spark

# Group by 'BOROUGH' and count the number of crashes
crashes_by_borough = sdf_small.groupBy("BOROUGH").agg(count("BOROUGH").alias("count")).orderBy(col("count").desc())

# Show the top 5 results
crashes_by_borough.show(5)




+---------+------+
|  BOROUGH| count|
+---------+------+
|         |131800|
| BROOKLYN| 92818|
|   QUEENS| 78333|
|MANHATTAN| 65046|
|    BRONX| 43378|
+---------+------+
only showing top 5 rows



 # Question 3

 > What is the most common cause of accident in 'QUEENS'

In [23]:
nyc_small[(nyc_small.BOROUGH == 'QUEENS')]['CONTRIBUTING FACTOR VEHICLE 1'].value_counts()

# you can also use a group by (to avoid the pandas value_counts function)

nyc_small[(nyc_small.BOROUGH == 'QUEENS')].groupby(
    'CONTRIBUTING FACTOR VEHICLE 1'
)['CONTRIBUTING FACTOR VEHICLE 1'].count().sort_values(ascending=False).head(5)

,CONTRIBUTING FACTOR VEHICLE 1
CONTRIBUTING FACTOR VEHICLE 1,
Unspecified,27394
Driver Inattention/Distraction,16605
Failure to Yield Right-of-Way,6682
Backing Unsafely,3853
Following Too Closely,2804


In [24]:
# Filter for accidents in 'QUEENS'
queens_accidents = sdf_small.filter(sdf_small['BOROUGH'] == 'QUEENS')

from pyspark.sql.functions import col

# Group by 'CONTRIBUTING FACTOR VEHICLE 1' and count the occurrences
common_causes_in_queens = queens_accidents.groupBy('CONTRIBUTING FACTOR VEHICLE 1') \
    .count().orderBy(col('count').desc())

# Show the top 5 most common causes
common_causes_in_queens.show(5)




+-----------------------------+-----+
|CONTRIBUTING FACTOR VEHICLE 1|count|
+-----------------------------+-----+
|                  Unspecified|27394|
|         Driver Inattentio...|16605|
|         Failure to Yield ...| 6682|
|             Backing Unsafely| 3853|
|         Following Too Clo...| 2804|
+-----------------------------+-----+
only showing top 5 rows



# Question 4

> _What is the average number or injuries for specific cars driving in specific suburbs_


In [25]:
nyc_small.groupby(['VEHICLE TYPE CODE 1', 'BOROUGH'])['NUMBER OF PERSONS INJURED'].mean().sort_values(ascending=False).head(3)

,,NUMBER OF PERSONS INJURED
VEHICLE TYPE CODE 1,BOROUGH,
bobct,BROOKLYN,5.0
PICKUP,BRONX,4.0
BACKH,QUEENS,4.0


In [26]:
from pyspark.sql.functions import avg

# Group by 'VEHICLE TYPE CODE 1' and 'BOROUGH' and calculate the average number of injuries
avg_injuries_by_vehicle_borough = sdf_small.groupBy('VEHICLE TYPE CODE 1', 'BOROUGH') \
    .agg(avg('NUMBER OF PERSONS INJURED').alias('avg_injuries')) \
    .orderBy('avg_injuries', ascending=False)

# Show the top 3 results
avg_injuries_by_vehicle_borough.show(3)


+-------------------+-------+------------+
|VEHICLE TYPE CODE 1|BOROUGH|avg_injuries|
+-------------------+-------+------------+
|         School bus|       |         6.0|
|            mta bus|       |         6.0|
|         DELIVERY T|       |         6.0|
+-------------------+-------+------------+
only showing top 3 rows

